# Main Code Space

# 1. Data Loading

In [5]:
from processing import *
import matplotlib.pyplot as plt
import numpy as np

folder_paths = ['DataFile/data_250612', 'DataFile/data_250616', 'DataFile/data_250618']
all_trials = []

for folder_path in folder_paths:
    trials = Load_files(folder_path, pattern='cropped_*.h5')
    trials = [trial for trial in trials if 'test' not in trial['file']]     # Temporary remove Test set
    all_trials.extend(trials)

trial_num = len(all_trials)
print(f"Total trials: {trial_num}")

Total 4 files are now loaded...
Total 98 trials are completely loaded !!
Total 5 files are now loaded...
Total 145 trials are completely loaded !!
Total 5 files are now loaded...
Total 200 trials are completely loaded !!
Total trials: 443
